In [1]:
!pip uninstall torch -y
!pip uninstall torch-scatter -y
!pip uninstall torch-sparsse -y
!pip uninstall torchvision -y
!pip uninstall torchaudio -y
!pip uninstall torch-spline-con -y
!pip uninstall torch-cluster -y
!pip uninstall torch-geometric -y

Found existing installation: torch 1.7.0
Uninstalling torch-1.7.0:
  Successfully uninstalled torch-1.7.0
Found existing installation: torchvision 0.8.1
Uninstalling torchvision-0.8.1:
  Successfully uninstalled torchvision-0.8.1
Found existing installation: torchaudio 0.7.0a0+ac17b64
Uninstalling torchaudio-0.7.0a0+ac17b64:
  Successfully uninstalled torchaudio-0.7.0a0+ac17b64


In [2]:
!pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html --target=/kaggle/working

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 1156.8 MB 6.6 kB/s 
     |████████████████████████████████| 12.9 MB 39.9 MB/s 
     |████████████████████████████████| 7.6 MB 701 kB/s 
     |████████████████████████████████| 15.7 MB 22 kB/s 
     |████████████████████████████████| 3.0 MB 22.1 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-cloud 0.1.13 requires tensorflow<3.0,>=1.15.0, which is not installed.
fancyimpute 0.5.5 requires tensorflow, which is not installed.
dask-cudf 21.6.1+2.g101fc0fda4 requires cupy-cuda112, which is not installed.
cudf 21.6.1+2.g101fc0fda4 requires cupy-cuda110, which is not installed.
yellowbrick 1.3.post1 requires numpy<1.20,>=1.16.0, but you have numpy 1.21.1 which is incompatible.
tensorflow-gpu 2.4.1 requires numpy~=1.19.2, but you have numpy 1.21

In [3]:
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.7.1+cu110.html --target=/kaggle/working
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.7.1+cu110.html --target=/kaggle/working
!pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-1.7.0+cu110.html --target=/kaggle/working
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-1.7.1+cu110.html --target=/kaggle/working
!pip install torch-geometric --target=/kaggle/working

Looking in links: https://pytorch-geometric.com/whl/torch-1.7.1+cu110.html
     |████████████████████████████████| 2.7 MB 1.6 MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.7.1+cu110.html
    ERROR: Command errored out with exit status 1:
     command: /opt/conda/bin/python3.7 -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-ziwbzkok/torch-sparse_84708b129a7d47bcbf6b5c0dde8157e4/setup.py'"'"'; __file__='"'"'/tmp/pip-install-ziwbzkok/torch-sparse_84708b129a7d47bcbf6b5c0dde8157e4/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /tmp/pip-pip-egg-info-y4vpibwq
         cwd: /tmp/pip-install-ziwbzkok/torch-sparse_84708b129a7d47bcbf6b5c0dde8157e4/
    Complete output (5 lines):
    Traceback (most rec

In [4]:
# refresh before running
import sys
sys.path.append('/kaggle/working')
# !ls /kaggle/working
import torch
print(torch.__version__)
print(torch.version.cuda)

1.7.1+cu110
11.0


In [5]:
from typing import Union, Tuple
from torch_geometric.typing import OptPairTensor, Adj, Size # Optional[Tensor], Union[Tensor, SparseTensor], Optional[Tuple[int, int]], all about data type

from torch import Tensor
from torch.nn import Linear
import torch.nn.functional as F
from torch_sparse import SparseTensor, matmul
from torch_geometric.nn.conv import MessagePassing

In [6]:
import warnings
warnings.filterwarnings("ignore")
import os
import time
import random
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from joblib import Parallel, delayed

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

import torch
import torch.nn.functional as F
import torch_geometric.nn as gnn
from torch_geometric.nn import MessagePassing
from torch_geometric.data import Dataset, Data, DataLoader
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
from torch_geometric.utils import softmax

In [7]:
inv = 0

In [8]:
df = pd.read_csv('../input/cms-dataset/CMS_trigger.csv')
print(df.columns)
df

scaler_1 = StandardScaler()
df.loc[:,'Phi_0':'MedianTheta'] = scaler_1.fit_transform(df.loc[:,'Phi_0':'MedianTheta']) # normalize

Index(['Unnamed: 0', 'Phi_0', 'Phi_2', 'Phi_3', 'Phi_4', 'Theta_0', 'Theta_2',
       'Theta_3', 'Theta_4', 'BendingAngle_0', 'BendingAngle_2',
       'BendingAngle_3', 'BendingAngle_4', 'TimeInfo_0', 'TimeInfo_2',
       'TimeInfo_3', 'TimeInfo_4', 'RingNumber_0', 'RingNumber_2',
       'RingNumber_3', 'RingNumber_4', 'Front_0', 'Front_2', 'Front_3',
       'Front_4', 'Mask_0', 'Mask_2', 'Mask_3', 'Mask_4',
       'PatternStraightness', 'Zone', 'MedianTheta', 'q/pt', 'PhiAngle',
       'EtaAngle'],
      dtype='object')


In [9]:
features = ['Phi_'+str(i) for i in [0,2,3,4]] + ['Theta_'+str(i) for i in [0,2,3,4]] + \
['Front_'+str(i) for i in [0,2,3,4]] + ['BendingAngle_'+str(i) for i in [0,2,3,4]] + \
['RingNumber_'+str(i) for i in [0,2,3,4]]
['TimeInfo_'+str(i) for i in [0,2,3,4]] + ['Mask_'+str(i) for i in [0,2,3,4]] #+ ['PatternStraightness'] + ['Zone'] + ['MedianTheta']
# edge_index = torch.tensor([(0,1),(1,2),(2,3),(3,2),(2,1),(1,0)], dtype=torch.long).T
edge_index = [(0,1),(1,2),(2,3),(3,2),(2,1),(1,0)]
# edge_index = [(0,1),(1,2),(2,3),(2,4),(1,0),(2,1),(3,2),(4,2)]
# edge_index = [(0,1),(1,2),(2,3),(2,4),(1,0),(0,2),(2,1),(3,2),(4,2),(2,5),(2,6),(5,2),(6,2)]
# edge_index = [(0,1),(1,2),(2,3),(2,4),(1,0),(2,1),(3,2),(4,2),(3,4),(4,3),(1,3),(1,4),(0,3),(0,4),(0,2)]

In [10]:
# x_train, x_test, pT_tr, pT_ts, inv_pT_tr, inv_pT_ts = train_test_split(df[features].to_numpy(), abs(1/df.loc[:,'q/pt']).to_numpy(), 1/abs(1/df.loc[:,'q/pt']).to_numpy(), test_size = 0.2, random_state = 1)
train_mask, test_mask = train_test_split(df['Unnamed: 0'].to_numpy(), test_size = 0.2, random_state = 1)
x_data = df[features].to_numpy()
pT = abs(1/df.loc[:,'q/pt']).to_numpy()
inv_pT = 1/pT
if inv:
    label = inv_pT
else:
    label = pT
num_features = x_data.shape[-1]
print('Data shape: ' + str(x_data.shape))
print(pT.shape)
print('Len train: '+str(len(train_mask))+', Len test: '+str(len(test_mask)))
print('Num. features: '+str(num_features))

Data shape: (1179356, 20)
(1179356,)
Len train: 943484, Len test: 235872
Num. features: 20


In [11]:
def process_data(i):
    
    data = Data(x=torch.tensor(x_data[i].reshape(-1,4).T, dtype=torch.float), y=torch.tensor(label[i], dtype=torch.float),
                edge_index=torch.tensor(edge_index, dtype = torch.long).T)
    return data

In [12]:
class MPL(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(MPL, self).__init__(aggr='add')
        self.mlp1 = torch.nn.Linear(in_channels*2, out_channels)
        self.mlp2 = torch.nn.Linear(in_channels, out_channels)
        self.mlp3 = torch.nn.Linear(2*out_channels, 1)
        self.mlp4 = torch.nn.Linear(2*out_channels, 1)
        self.mlp5 = torch.nn.Linear(in_channels,16)
        self.mlp6 = torch.nn.Linear(out_channels,16)
        self.mlp7 = torch.nn.Linear(16,1)

    def forward(self, x, edge_index):
#         edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        msg = self.propagate(edge_index, x=x)
        x = F.relu(self.mlp2(x))
        w1 = F.sigmoid(self.mlp3(torch.cat([x,msg], dim=1)))
        w2 = F.sigmoid(self.mlp4(torch.cat([x,msg], dim=1)))
        out = w1*msg + w2*x
        
        return out

    def message(self, x_i, x_j, edge_index):
        msg = F.relu(self.mlp1(torch.cat([x_i, x_j-x_i], dim=1)))
        w1 = F.tanh(self.mlp5(x_i))
        w2 = F.tanh(self.mlp6(msg))
        w = self.mlp7(w1*w2)
        w = softmax(w, edge_index[0])
        return msg*w

In [13]:
class MPNN(torch.nn.Module):
    def __init__(self):
        super(MPNN, self).__init__()
        self.conv1 = MPL(int(len(features)/4),128 )
        self.conv2 = MPL(128,64)
        self.conv3 = MPL(64,64 )
        self.conv4 = MPL(64,64 )
        self.lin1 = torch.nn.Linear(128, 128)
        self.lin2 = torch.nn.Linear(128, 16)
        self.lin3 = torch.nn.Linear(16, 16)
        self.lin4 = torch.nn.Linear(16, 1)
        self.lin5 = torch.nn.Linear(128, 128)
        self.lin6 = torch.nn.Linear(128, 16)
        self.lin7 = torch.nn.Linear(16, 16)
        self.lin8 = torch.nn.Linear(16, 1)
        self.global_att_pool1 = gnn.GlobalAttention(torch.nn.Sequential(torch.nn.Linear(64, 1)))
        self.global_att_pool2 = gnn.GlobalAttention(torch.nn.Sequential(torch.nn.Linear(64, 1)))
    
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x1 = self.global_att_pool1(x, batch)
        x = F.relu(self.conv3(x, edge_index))
        x = F.relu(self.conv4(x, edge_index))
        x2 = self.global_att_pool2(x, batch)
        x_out = torch.cat([x1, x2], dim=1)
        x = F.relu(self.lin1(x_out))
        x = F.relu(self.lin2(x))
        x = self.lin3(x)
        x = self.lin4(x).squeeze(1)

        return x

In [14]:
class MyDataset(Dataset):
    def __init__(self, indices=list(range(len(df))), transform=None):
        self.transform = transform
        self.indices = indices
    
    def __getitem__(self, idx):
        return process_data(self.indices[idx])
    
    def __len__(self):
        return len(self.indices)

In [15]:
def pTLossTorch(y_pred,y_true):
#     if not inv:
#         y_pred = torch.pow(y_pred,-1)
#         y_true = torch.pow(y_true,-1)
    y_t = (y_true<80).type(torch.FloatTensor)*y_true.type(torch.FloatTensor) + (y_true>=80).type(torch.FloatTensor)*(y_true<250).type(torch.FloatTensor)*y_true.type(torch.FloatTensor)**2.4 + (y_true>=160).type(torch.FloatTensor)*10 
    return torch.mean(y_t.type(torch.FloatTensor)*torch.pow((y_pred-y_true)/y_true,2).type(torch.FloatTensor))/250

In [16]:
def mse_custom(outputs, labels):
    weights = torch.tensor(labels<80, dtype=torch.float).to(device)*labels + torch.tensor(labels>=80, dtype=torch.float).to(device)*torch.tensor(labels<160, dtype=torch.float).to(device)*labels*2.4 + torch.tensor(labels>=160, dtype=torch.float).to(device)*10
    error = weights*(((outputs-labels)/labels)**2)
    return torch.mean(error)

In [17]:
def train(prog_bar = True):
    
    train_losses, test_losses = list(), list()
    min_test_loss = float('inf')
    train_loader = DataLoader(MyDataset(indices=train_mask), batch_size=batch_size)
    test_loader = DataLoader(MyDataset(indices=test_mask), batch_size=batch_size)
    
    for epoch in range(epochs):
        train_loss = 0
        test_loss = 0
        if prog_bar:
            pbar = tqdm(train_loader,position=0)
        else:
            pbar = train_loader
            
        # train
        for data in pbar:
            data = data.to(device)
            optimizer.zero_grad()
            out = model(data)
            labels = data.y
            loss = pTLossTorch(out, data.y)
            loss.backward()
            optimizer.step()
            if prog_bar:
                pbar.set_description('pTLoss: '+str(loss.cpu().detach().numpy()))
                train_loss += loss.cpu().detach().numpy()/len(train_loader)
                
        # test
        for data in test_loader:
            data = data.to(device)
            optimizer.zero_grad()
            out = model(data)
            labels = data.y
            loss = pTLossTorch(out, data.y)
            test_loss += loss.cpu().detach().numpy()/len(test_loader)
        if test_loss<min_test_loss:
            print('Min loss changed from '+str(min_test_loss)+' to '+str(test_loss))
            min_test_loss = test_loss
            torch.save(model.state_dict(), model_name)
        train_losses.append(train_loss)
        test_losses.append(test_loss)
        if epoch > 10 and min(test_losses[-7:])>min_test_loss+1e-9:
            break
        lr_scheduler.step(test_loss)
        print('Epoch: ', str(epoch+1)+'/'+str(epochs),'| Training pTLoss: ', train_loss, '| Testing pTLoss: ', test_loss)
        
        if not prog_bar:
            plt.plot(train_losses, label="Train Loss")
            plt.plot(test_losses, label="Validation Loss")
            plt.xlabel("# Epoch")
            plt.ylabel("Loss")
            plt.legend(loc='upper right')
            plt.show()
    return train_losses, test_losses

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
batch_size = 2**12
epochs = 50
model = MPNN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.002, weight_decay=5e-4)
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, verbose=True, patience=1, factor=0.5)

model_name = 'GNN_v10_' + 'inv_' + str(inv) + '.pth'
train_losses, test_losses = train(prog_bar=True)

  0%|          | 0/231 [00:00<?, ?it/s]

Min loss changed from inf to 2.478362739086151
Epoch:  1/50 | Training pTLoss:  5.362514843672387 | Testing pTLoss:  2.478362739086151


  0%|          | 0/231 [00:00<?, ?it/s]

Min loss changed from 2.478362739086151 to 2.137275556038166
Epoch:  2/50 | Training pTLoss:  2.446959033156887 | Testing pTLoss:  2.137275556038166


  0%|          | 0/231 [00:00<?, ?it/s]

...

Min loss changed from 1.5148916737786657 to 1.5124112038776791
Epoch:  48/50 | Training pTLoss:  1.4132668061173834 | Testing pTLoss:  1.5124112038776791


  0%|          | 0/231 [00:00<?, ?it/s]

Min loss changed from 1.5124112038776791 to 1.5123664017381342
Epoch:  49/50 | Training pTLoss:  1.4119040674461425 | Testing pTLoss:  1.5123664017381342


  0%|          | 0/231 [00:00<?, ?it/s]

Min loss changed from 1.5123664017381342 to 1.5122750463156862
Epoch    50: reducing learning rate of group 0 to 1.9531e-05.
Epoch:  50/50 | Training pTLoss:  1.4114317300515775 | Testing pTLoss:  1.5122750463156862


In [19]:
model_name = './GNN_v10_inv_' + str(inv) + '.pth'
batch_size = 2**12
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
loaded_model = MPNN().to(device)
rand_model = MPNN().to(device)
loaded_model.load_state_dict(torch.load(model_name))

<All keys matched successfully>

In [20]:
test_loader = DataLoader(MyDataset(indices=test_mask), batch_size=batch_size)
# test
test_los = 0
preds = []
for data in tqdm(test_loader,position=0):
    data = data.to(device)
    out = model(data) # change rand_model to seee if it's trained - should not be the same loss
    preds.append(out.cpu().detach())
    labels = data.y
    loss = pTLossTorch(out, data.y)
    test_los += loss.cpu().detach().numpy()/len(test_loader)
print('Test_loss: '+str(test_los))

  0%|          | 0/58 [00:00<?, ?it/s]

Test_loss: 1.5122750463156862


In [21]:
# save to csv
pred_ls = [float(i) for p in preds for i in p]
len(pred_ls)
df_pred = pd.DataFrame(pred_ls)
df_pred.to_csv('./GNN_v10_pTLoss_inv_' + str(inv) + '_7feats.csv')